In [ ]:
import src.datasets.tabular
import src.datasets.utils
from src.datasets.tabular import get_dataset_config

## preprocessing for different datasets to vw format
dataset = 'adult'
dataset_kwargs = {'root_dir': 'src/datasets'}
dataset_config = get_dataset_config(dataset, **dataset_kwargs)
dset = src.datasets.utils.get_dataset(dataset_config)
X_tr, y_tr, __, __, __, __ =  dset.get_data()

print(sum(y_tr), len(y_tr))

src.datasets.utils.save_vw_dataset(X_tr, y_tr, dataset, 'src/datasets/adult')


In [ ]:
import subprocess, sys, os
VW = 'vw'
ds = 'src/datasets/adult/adult_2.vw.gz'
cmd = [VW, ds, '-b', '24']
# supervised
# cmd += ['--oaa', '2']
# contextual bandit
## --progress 1 means we output all the time loss, otherwise it will only show the time point pow(2,k) 
## param details of different kinds of cb algs are in run_vw_job.py, i.e. the process function.
cmd += ['--cbify', '2', '--bag', '4', '--progress', '1']
output = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
#output = os.system('vw ' + ds + ' -b 24 --oaa 2') 
output = str(output, encoding = 'utf-8')
from src.utils.joint_regret import output_extraction
#the return of output_extraction function is the cummlative loss in terms of the time
cum_loss = output_extraction(output)
cum_loss


In [ ]:
!vw src/datasets/adult/adult_2.vw.gz 24 --oaa 2

In [ ]:
!vw src/datasets/german/german_2.vw.gz -b 24 --cbify 2 --regcbopt

In [1]:
## the falcon procedure, for the paper
## https://arxiv.org/pdf/2003.12699.pdf
from src.models.cb.FALCON import FALCON, FALCON_ldf, FALCON_price
#from split_gz_csv import ds_files_csv, ds_files
#from split import sample_custom_pmf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
#from glob_param import *
from src.run_vw_job import process
#from split import *
import re
rgx = re.compile('^average loss = (.*)$', flags=re.M)
#show the regret performance in each algorithm

csv_path = 'src/datasets/adult/adult_2.csv'
# def alg_performance_regret(DS_DIR, did, alg_param, did_type):
loss_param = []
    # if did_type == 'openml':
    #     sample_falcon = FALCON(csv_path, 95, 1)
    # elif did_type == 'ms':
    #     sample_falcon = FALCON_ldf(csv_path, 105, 1, 10)
    # elif did_type == 'yahoo':
    #     sample_falcon = FALCON_ldf(csv_path, 105, 1, 6, 'ridge')
    # elif did_type == 'loan':
    #     if did == 0:
    #         sample_falcon = FALCON_price(csv_path, 10, 1, 15)
    #     elif did == 1:
    #         sample_falcon = FALCON_price(csv_path, 10, 1, 10)
#gamma_param is a hyperparam in FALCON alg.
sample_falcon = FALCON(csvpath = csv_path, gamma_param = 95, feed_choice = 1,
                        group = ['sex', 'race'])
    
sample_falcon.learn_schedule()

#cummulative loss for FALCON alg.
sample_falcon.loss_all
        
    # gz_path = ds_files(DS_DIR)[did]
    # for index in range(len(alg_param)):
    #     pv_loss, output = process(gz_path, alg_param[index], None, True, did_type)
    #     loss_param.append(list(output_extraction(output)))
    # return loss_param

3


array([0., 0., 0., ..., 0., 0., 0.])

In [2]:
# an example to test whether the eval metric works well in FALCON
from src.eval_metric_cb import offline_eval_metric_cb
eval_falcon = offline_eval_metric_cb(group = sample_falcon.group, context = sample_falcon.context_all,
                              action = sample_falcon.action_all, loss = sample_falcon.loss_all)
eval_falcon.group_loss_parity()

/Users/wty6162/Desktop/fa-alg/src/eval_metric_cb.py:46: RuntimeWarning: invalid value encountered in double_scalars
  avg_loss = [cumu_group_loss[i][k] / cumu_group_num[i][k] for k in range(self.group_num)]


{'cumu_loss': 5952.0,
 'cumu_loss_0': 4342.0,
 'cumu_loss_1': 491.0,
 'cumu_loss_2': 174.0,
 'cumu_loss_3': 945.0,
 'loss_parity': 0.14472397670577375}

In [3]:
eval_falcon.group_action_parity()

/Users/wty6162/Desktop/fa-alg/src/eval_metric_cb.py:91: RuntimeWarning: invalid value encountered in double_scalars
  group_action_parity[k] = np.sum([abs(avg_action_num[j]/(i + 1) - group_action_num[k][j]/sum(group_action_num[k]))


In [4]:
eval_falcon.summary_loss

{'cumu_loss': 5952.0,
 'cumu_loss_0': 4342.0,
 'cumu_loss_1': 491.0,
 'cumu_loss_2': 174.0,
 'cumu_loss_3': 945.0,
 'loss_parity': 0.14472397670577375,
 'group_action_parity_realized': 0.33976848079153216}

In [ ]:
output_all = output.split('\n')
res = []
for i in range(len(output_all)):
    if len(output_all[i].split(' ')) >= 50:
        res.append(float(output_all[i].split(' ')[0]))
        #print(len(output_all[i].split(' ')))
    #print(len(output_all[i].split(' ')))
len(res)

In [ ]:
df = pd.read_csv('src/datasets/adult/adult_2.csv')
b = list(np.array(df[['sex', 'race']]))

In [ ]:
idx_num = 0
set_idx = {}
c = np.zeros(len(df))
for i in range(len(df)):
    str_key = str(b[i][0]) + ',' + str(b[i][1])
    if str_key not in set_idx:
        set_idx[str_key] = idx_num
        idx_num += 1
    c[i] = set_idx[str_key]

c